In [1]:
from pynwb import NWBHDF5IO
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import datetime
import scipy
from scipy import signal
import multiprocessing
import fhMethods
import cusignal
import cupy as cp
import itertools
from tqdm import tqdm

# Device 0 and Device 1 are K80s
cp.cuda.Device(0).use() #- K80 Unit 1
#cp.cuda.Device(1).use() - K80 Unit 2


filename = r"E:\data\6305\patient6305.nwb"
nwbIO = NWBHDF5IO(filename, 'r')
openNWB = nwbIO.read()

ecephys = openNWB.processing['ecephys']
(samples, signalNumber) = np.shape(openNWB.processing['ecephys'].data_interfaces['LAH'].data[:,:])
samplingRate = ecephys.data_interfaces['LAH'].rate

In [2]:
recordingStartTimestamp = openNWB.session_start_time.timestamp()
seizureList = []

#Seizure 1
#seizureStart = datetime.datetime(2020, 3, 5, 5, 30, 30) long
seizureStart = datetime.datetime(2020, 3, 5, 7, 0, 0)
seizureOnsetStart = datetime.datetime(2020, 3, 5, 7, 1, 7)
seizureOnsetEnd = datetime.datetime(2020, 3, 5, 7, 2, 53)
seizureEnd = datetime.datetime(2020, 3, 5, 7, 3, 30)
#seizureEnd = datetime.datetime(2020, 3, 5, 7, 40, 30)

seizureTsStart = seizureStart.timestamp()
seizureTsEnd = seizureEnd.timestamp()

seizureTsOnsetStart = seizureOnsetStart.timestamp()
seizureTsOnsetEnd = seizureOnsetEnd.timestamp()


kadStart = seizureTsStart - recordingStartTimestamp
kadEnd = seizureTsEnd - recordingStartTimestamp
onsetStartTs = seizureTsOnsetStart - recordingStartTimestamp
onsetEndTs = seizureTsOnsetEnd - recordingStartTimestamp

kStart = int(kadStart * samplingRate)
kEnd = int(kadEnd * samplingRate)
oStart = int(onsetStartTs * samplingRate)
oEnd = int(onsetEndTs * samplingRate)

seizureList.append((kStart, kEnd, oStart, oEnd))



In [ ]:
#Seizure 2
seizureStart = datetime.datetime(2020, 3, 5, 6, 20, 30)
seizureOnsetStart = datetime.datetime(2020, 3, 5, 7, 50, 30)
seizureOnsetEnd = datetime.datetime(2020, 3, 5, 7, 52, 43)
seizureEnd = datetime.datetime(2020, 3, 5, 9, 30, 30)

seizureTsStart = seizureStart.timestamp()
seizureTsEnd = seizureEnd.timestamp()

seizureTsOnsetStart = seizureOnsetStart.timestamp()
seizureTsOnsetEnd = seizureOnsetEnd.timestamp()


kadStart = seizureTsStart - recordingStartTimestamp
kadEnd = seizureTsEnd - recordingStartTimestamp
onsetStartTs = seizureTsOnsetStart - recordingStartTimestamp
onsetEndTs = seizureTsOnsetEnd - recordingStartTimestamp


kStart = int(kadStart * samplingRate)
kEnd = int(kadEnd * samplingRate)
oStart = int(onsetStartTs * samplingRate)
oEnd = int(onsetEndTs * samplingRate)
seizureList.append((kStart, kEnd, oStart, oEnd))

#Seizure 3
seizureStart = datetime.datetime(2020, 3, 5, 7, 30, 11)
seizureOnsetStart = datetime.datetime(2020, 3, 5, 9, 3, 11)
seizureOnsetEnd = datetime.datetime(2020, 3, 5, 9, 5, 3)
seizureEnd = datetime.datetime(2020, 3, 5, 10, 30, 30)

seizureTsStart = seizureStart.timestamp()
seizureTsEnd = seizureEnd.timestamp()

seizureTsOnsetStart = seizureOnsetStart.timestamp()
seizureTsOnsetEnd = seizureOnsetEnd.timestamp()


kadStart = seizureTsStart - recordingStartTimestamp
kadEnd = seizureTsEnd - recordingStartTimestamp
onsetStartTs = seizureTsOnsetStart - recordingStartTimestamp
onsetEndTs = seizureTsOnsetEnd - recordingStartTimestamp


kStart = int(kadStart * samplingRate)
kEnd = int(kadEnd * samplingRate)
oStart = int(onsetStartTs * samplingRate)
oEnd = int(onsetEndTs * samplingRate)
seizureList.append((kStart, kEnd, oStart, oEnd))


In [ ]:
seizureList.pop(1)

In [ ]:
seizureList

In [3]:
contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
contactCombos = list(itertools.combinations(contactsOfInterest, 2))

In [ ]:
car = carData.data_interfaces['Common Average Reference'].data[:]

In [ ]:
regions = list(ecephysData.data_interfaces.keys())
regions.pop(0) # remove EKG

In [5]:
%matplotlib widget
fs = samplingRate       # Sample rate, Hz
cutoff = 3    # Desired cutoff frequency, Hz
trans_width = 0.01  # Width of transition from pass band to stop band, Hz
numtaps = 1001    # Size of the FIR filter.
taps = signal.firls(numtaps, [0, cutoff - trans_width, cutoff, 0.5*fs], [0, 0, 1, 1], fs=fs)
w, h = signal.freqz(taps, [1], worN=2000)

In [ ]:
# First, calculate the fhData so we can run the comparisons.
fhObjects = []
for (idx, regionAndContact) in enumerate((contactsOfInterest)):
    thisContactSeizures = []
    for (jdx, seizure) in enumerate(seizureList):
        (kStart, kEnd, oStart, oEnd) = seizure # unpack the tuple
        regionOfInterest = regionAndContact[0]
        contact = regionAndContact[1]
        thisContactSeizures.append((fhMethods.FilterHilbert(), seizure)) # include the timestamps of the data.
        thisContactSeizures[jdx][0].generateFilterArray(1, samplingRate, 4)
        dat = (np.single(ecephysData.data_interfaces[regionOfInterest].data[kStart:kEnd, contact]))
        dat = signal.lfilter(taps, [1],dat)
        #plt.figure()
        #plt.specgram(dat, Fs=samplingRate)
        #onset  = int(oStart - kStart)/samplingRate
        #offset = int(oEnd - kStart)/samplingRate
        #plt.vlines(int(onset), ymin=0, ymax=100, color='r')
        #plt.vlines(int(offset), ymin=0, ymax=100, color='r')
        ##plt.vlines(int(onset), ymin=np.min(dat), ymax=np.max(dat), color='r')
        ##plt.vlines(int(offset), ymin=np.min(dat), ymax=np.max(dat), color='r')
        #plt.title(regionOfInterest + str(contact)+' seizure ' + str(jdx))
        #plt.savefig(regionOfInterest + str(contact)+' seizure ' + str(jdx) + ' specgram.png')
        #thisContactSeizures[jdx][0].runFilterHilbert(dat)
    fhObjects.append(thisContactSeizures) # put all fhObjects into a list of lists.

In [ ]:
fhObjects

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][0][0].fhResult, fhObjects[0][0][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPH 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][1][0].fhResult, fhObjects[0][1][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPH 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][2][0].fhResult, fhObjects[0][2][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPH 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][2][0].fhResult, fhObjects[4][2][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPI 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][1][0].fhResult, fhObjects[4][1][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPI 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][0][0].fhResult, fhObjects[4][0][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs LPI 5")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][0][0].fhResult, fhObjects[2][0][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs RAH 3")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][1][0].fhResult, fhObjects[2][1][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs RAH 3")

In [ ]:
# contactsOfInterest = [('LPH', 5), ('LPH', 6), ('RAH', 3), ('RAH', 4), ('LPI', 5), ('LPI', 6),('LAI',5), ('LAI',6), ('LTP', 1), ('LTP', 6), ('LTP', 7), ('LTP', 8)]
seizureTimes = fhObjects[0][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
onset = oStart - kStart
offset = oEnd - kStart
compareRegionsAndChannels(fhObjects[0][0][0], fhObjects[1][2][0].fhResult, fhObjects[2][2][0].fhResult, samplingRate, onset, offset)
plt.title("LPH 6 vs RAH 3")

In [ ]:
# Get TS data to plot raw trace
seizureTimes = fhObjects[1][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph6Seizure1 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 6] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph6Seizure2 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 6] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph6Seizure3 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 6] - car[kStart:kEnd]))

# LPH 5
seizureTimes = fhObjects[1][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph5Seizure1 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph5Seizure2 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lph5Seizure3 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPH'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

# LPI 5
seizureTimes = fhObjects[1][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lpi5Seizure1 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPI'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lpi5Seizure2 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPI'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
lpi5Seizure3 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['LPI'].data[kStart:kEnd, 5] - car[kStart:kEnd]))

# RAH 3
seizureTimes = fhObjects[1][0][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
rah3Seizure1 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['RAH'].data[kStart:kEnd, 3] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][1][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
rah3Seizure2 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['RAH'].data[kStart:kEnd, 3] - car[kStart:kEnd]))

seizureTimes = fhObjects[1][2][1]
(kStart, kEnd, oStart, oEnd) = seizureTimes
rah3Seizure3 = signal.lfilter(taps, [1], (ecephysData.data_interfaces['RAH'].data[kStart:kEnd, 3] - car[kStart:kEnd]))

In [ ]:
lph6Complete = (np.single(ecephysData.data_interfaces['LPH'].data[:, 5] - car))
lph6Complete = signal.lfilter(taps, [1],lph6Complete)

lph6fh = fhMethods.FilterHilbert()
lph6fh.generateFilterArray(1, samplingRate, 4)
lph6fh.runFilterHilbert(lph6Complete)

lpi5Complete = (np.single(ecephysData.data_interfaces['LPI'].data[:, 5] - car))
lpi5Complete = signal.lfilter(taps, [1],lph6Complete)

lpi5fh = fhMethods.FilterHilbert()
lpi5fh.generateFilterArray(1, samplingRate, 4)
lpi5fh.runFilterHilbert(lpi5Complete)

In [ ]:
kEnd-kStart

In [ ]:
compareRegionsAndChannels(lph6fh, np.csingle(lph6fh.fhResult), np.csingle(lpi5fh.fhResult), samplingRate, seizureList[0][2], seizureList[0][3])

In [ ]:
def compareRegionsAndChannels(fhObject, dat1, dat2, samplingRate, onset, offset):
    ppCoupling = fhObject.calculatePhaseDifferencesSlow(dat1, dat2, 10, 0.5)
    plt.figure()
    plt.pcolormesh(np.single(ppCoupling), rasterized=True)
    plt.vlines(int(onset), ymin=0, ymax=100, color='r')
    plt.vlines(int(offset), ymin=0, ymax=100, color='r')

In [ ]:
def plot_response(fs, w, h, title):
    "Utility function to plot response functions"
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(0.5*fs*w/np.pi, 20*np.log10(np.abs(h)))
    ax.set_ylim(-40, 5)
    ax.set_xlim(0, 0.5*fs)
    ax.grid(True)
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Gain (dB)')
    ax.set_title(title)